# Trading Crypto

In this notebook we're going to do a what-if analysis based on some trades of Cryptocurrencies. We're going to work out the valuation of our portfolio based on doing some trades and then re-running that valuation as if we hadn't made those trades

In [12]:
# Import LUSID
import lusid
import os
from lusidjam import RefreshingToken
import pandas as pd
import pytz
from datetime import datetime, timedelta
from dateutil.parser import parse

In [6]:
# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

In [8]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)
derived_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)

## Instruments Master

We've got some Cryptocurrency instruments in a CSV file. Let's import those into LUSID:

In [9]:
instruments_file = "data/instruments.csv"
instruments = pd.read_csv(instruments_file)
instruments

,instrument_name,id,currency,asset_class
0,Dogecoin,DOGE,USD,crypto
1,Bitcoin,BTC,USD,crypto
2,XRP,XRP,USD,crypto


In [ ]:
definitions = {}
for _, instrument in instruments.iterrows():
    identifiers = {
        "ClientInternal": lusid.models.InstrumentIdValue(value=instrument["id"])
    }
    definitions[instrument['instrument_name']] = lusid.models.InstrumentDefinition(
        name=instrument['instrument_name'], identifiers=identifiers)

instruments_api.upsert_instruments(request_body=definitions)

## Create Portfolio

Next we'll create a portfolio:

In [37]:
now = datetime.now().strftime('%Y-%m-%d-%H_%M_%S')
scope = portfolio_code = f"Developer-WhatIf-Tutorial-{now}"
print("Portfolio Code", portfolio_code)

created_date = datetime(year=2021, month=1, day=1, tzinfo=pytz.UTC)
response = transaction_portfolios_api.create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=lusid.models.CreateTransactionPortfolioRequest(
        display_name="Developer What If Tutorial",
        code=portfolio_code,
        created=created_date,
        base_currency="USD"))

Portfolio Code Developer-WhatIf-Tutorial-2021-05-06-11_01_58


## Transactions

And now let's load some transactions into that portfolio:

In [23]:
transactions_file = "data/transactions.csv"
transactions = pd.read_csv(transactions_file)
transactions

,txn_id,trade_date,transaction_type,instrument_desc,instrument_id,currency,quantity,price,net_money
0,tx_00001,2021-01-01,FundsIn,CASH_USD,cash,USD,1000,1.000000,1000.000000
1,tx_00002,2021-01-02,Buy,Dogecoin,DOGE,USD,100000,0.005405,540.480280
2,tx_00003,2021-02-02,Sell,Dogecoin,DOGE,USD,50000,0.031479,1573.959385
3,tx_00004,2021-02-02,Buy,XRP,XRP,USD,4000,0.368082,1472.328220


In [38]:
transactions_request = []
for row, txn in transactions.iterrows():
    if txn["instrument_id"] == "cash":
        instrument_identifier = {"Instrument/default/Currency": txn["currency"]}
    else:
        instrument_identifier = {"Instrument/default/ClientInternal": txn["instrument_id"]}

    transactions_request.append(
        lusid.models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=pytz.UTC.localize(parse(txn["trade_date"])),
            settlement_date=pytz.UTC.localize(parse(txn["trade_date"])),
            units=txn["quantity"],
            transaction_price=lusid.models.TransactionPrice(price=txn["price"], type="Price"),
            total_consideration=lusid.models.CurrencyAndAmount(
                amount=txn["net_money"], currency=txn["currency"])))

response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transactions_request)

## Quotes / Market Data

To run a valuation of a portfolio, we'll need to load in some quotes or market data. The CSV file described below contains the end of day prices for Dogecoin and XRP on 5th May 2021:

In [25]:
quotes_file = "data/quotes.csv"
quotes = pd.read_csv(quotes_file)
quotes

,instrument_name,id,date,price
0,Dogecoin,DOGE,2021-05-05,0.542511
1,XRP,XRP,2021-05-05,1.590046


We're going to load these into LUSID, but first we need to lookup the LUSID Instrument ID for each instrument:

In [49]:
response = instruments_api.get_instruments(
    identifier_type='ClientInternal',
    request_body=quotes["id"].values.tolist())
instruments = pd.DataFrame([{
    "id": instrument.identifiers["ClientInternal"],
    "luid": instrument.lusid_instrument_id}
    for _, instrument in response.values.items()
])
quotes_with_luid = pd.merge(quotes, instruments, on=["id"])
quotes_with_luid

,instrument_name,id,date,price,luid
0,Dogecoin,DOGE,2021-05-05,0.542511,LUID_5H4VJ0EE
1,XRP,XRP,2021-05-05,1.590046,LUID_HNFJOTFE


The default recipe that we'll use in the valuation section assumes that quotes are loaded using a LUSID Instrument ID, which we can do using the following code:

In [39]:
quotes_request = {
    f"quote_request_{quote['instrument_name']}_{quote['date']}": lusid.models.UpsertQuoteRequest(
        quote_id=lusid.models.QuoteId(
            quote_series_id=lusid.models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=quote["luid"],
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="mid",
            ),
            effective_at=pytz.UTC.localize(parse(quote['date'])).isoformat(),
        ),
        metric_value=lusid.models.MetricValue(value=quote['price'], unit="USD"),
    )
    for _, quote in quotes_with_luid.iterrows()
}

response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_request)

## Valuation


Now let's run a valuation, which will compute the value of our holdings using quotes/market data on a specified date. 

In [28]:
def compute_valuation_with_default_recipe(portfolio_code, from_date, to_date, metrics, group_by):
    return aggregation_api.get_valuation(
        valuation_request=lusid.models.ValuationRequest(
            recipe_id=lusid.models.ResourceId(scope=scope, code="default"),
            metrics=[lusid.models.AggregateSpec(key, op) for key, op in metrics],
            group_by=group_by,
            valuation_schedule=lusid.models.ValuationSchedule(effective_from=from_date, effective_at=to_date),
            portfolio_entity_ids=[lusid.models.PortfolioEntityId(
                scope=scope,
                code=portfolio_code,
                portfolio_entity_type="SinglePortfolio"
            )])).data

We've only loaded quotes for 5th May 2021, so we'll use that as our effective date:

In [30]:
effective_at = datetime(year=2021, month=5, day=5, tzinfo=pytz.UTC)

If we'd loaded more quotes we'd be able to track our portfolio's value over time.

We can run a valuation that returns the total value of our portfolio using the code below:

In [40]:
response = compute_valuation_with_default_recipe(
    portfolio_code,
    effective_at,
    effective_at,
    metrics=[
        ("Analytic/default/ValuationDate", "Value"),
        ("Holding/default/PV", "Sum"),
    ],
    group_by=["Analytic/default/ValuationDate"]
)
valuation_all = pd.DataFrame(response)
valuation_all

,Analytic/default/ValuationDate,Sum(Holding/default/PV)
0,2021-05-05T00:00:00.0000000+00:00,34046.883352


And if we want to get a breakdown by instrument we can do that as well:

In [41]:
effective_at = datetime(year=2021, month=5, day=5, tzinfo=pytz.UTC)
response = compute_valuation_with_default_recipe(
    portfolio_code,
    effective_at,
    effective_at,
    metrics=[
        ("Instrument/default/Name", "Value"),
        ("Holding/default/Cost", "Value"),
        ("Holding/default/Units", "Sum"),
        ("Holding/default/PV", "Sum"),
        ("Holding/default/PV", "Proportion")
    ],
    group_by=["Instrument/default/Name"]
)
valuation = pd.DataFrame(response)
valuation

,Instrument/default/Name,Holding/default/Cost,Sum(Holding/default/Units),Sum(Holding/default/PV),Proportion(Holding/default/PV)
0,USD,561.15,561.150885,561.150885,0.016482
1,Dogecoin,270.24,50000.000000,27125.548335,0.796712
2,XRP,1472.33,4000.000000,6360.184131,0.186807


## What if analysis

Now for the fun stuff! What if we didn't do those transactions on 2nd February 2021, where we sold half of our DOGE and bought XRP with the receipts?

We can work this out by following these three steps: 

* Create a derived portfolio of our initial portfolio
* Remove the 2nd February 2021 transactions from the derived portfolio
* Run the valuation on the derived portfolio

Our initial portfolio will remain untouched, but we'll re-run the valuation on our initial portfolio to make sure

In [42]:
derived_portfolio_code = f"Developer-WhatIf-Tutorial-Derived-{now}"

derived_request = lusid.models.CreateDerivedTransactionPortfolioRequest(
    display_name=f"Derived Portfolio of {portfolio_code}",
    code=derived_portfolio_code,
    parent_portfolio_id=lusid.models.ResourceId(scope=scope, code=portfolio_code),
    description="What if we didn't sell DOGE?",
    created=created_date
)

response = derived_api.create_derived_portfolio(
    scope=scope,
    create_derived_transaction_portfolio_request=derived_request
)

Let's delete the transactions on 2nd February 2021, which have the transactions IDs `tx_00003` and `tx_00004`:

In [ ]:
response = transaction_portfolios_api.cancel_transactions(
        scope=scope,
        code=derived_portfolio_code,
        transaction_ids=["tx_00003", "tx_00004"]
)

And now we can run a valuation on our initial portfolio and the derived one to see the difference that those trades have made on our portfolio's valuation:

In [47]:
for code in [portfolio_code, derived_portfolio_code]:
    effective_at = datetime(year=2021, month=5, day=5, tzinfo=pytz.UTC)
    response = compute_valuation_with_default_recipe(
        code,
        effective_at,
        effective_at,
        metrics=[
            ("Analytic/default/ValuationDate", "Value"),
            ("Holding/default/PV", "Sum"),
        ],
        group_by=["Analytic/default/ValuationDate"]
    )
    valuation_all = pd.DataFrame(response)
    print(f"Valuation for {code}")
    display(valuation_all)
    print("")

    response = compute_valuation_with_default_recipe(
        code,
        effective_at,
        effective_at,
        metrics=[
            ("Instrument/default/Name", "Value"),
            ("Holding/default/Cost", "Value"),
            ("Holding/default/Units", "Sum"),
            ("Holding/default/PV", "Sum"),
            ("Holding/default/PV", "Proportion")
        ],
        group_by=["Instrument/default/Name"]
    )
    valuation = pd.DataFrame(response)
    print(f"Valuation per instrument for {code}")
    display(valuation)
    print("")

Valuation for Developer-WhatIf-Tutorial-2021-05-06-11_01_58


,Analytic/default/ValuationDate,Sum(Holding/default/PV)
0,2021-05-05T00:00:00.0000000+00:00,34046.883352



Valuation per instrument for Developer-WhatIf-Tutorial-2021-05-06-11_01_58


,Instrument/default/Name,Holding/default/Cost,Sum(Holding/default/Units),Sum(Holding/default/PV),Proportion(Holding/default/PV)
0,USD,561.15,561.150885,561.150885,0.016482
1,Dogecoin,270.24,50000.000000,27125.548335,0.796712
2,XRP,1472.33,4000.000000,6360.184131,0.186807



Valuation for Developer-WhatIf-Tutorial-Derived-2021-05-06-11_01_58


,Analytic/default/ValuationDate,Sum(Holding/default/PV)
0,2021-05-05T00:00:00.0000000+00:00,54710.61639



Valuation per instrument for Developer-WhatIf-Tutorial-Derived-2021-05-06-11_01_58


,Instrument/default/Name,Holding/default/Cost,Sum(Holding/default/Units),Sum(Holding/default/PV),Proportion(Holding/default/PV)
0,USD,459.52,459.51972,459.51972,0.008399
1,Dogecoin,540.48,100000.00000,54251.09667,0.991601
